In [13]:
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI

In [14]:
import os

In [15]:
# OpenAI API configuration
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [22]:
neo4j_url = "bolt://localhost:7687"
neo4j_user = "neo4j"
neo4j_password = "12345678"

In [23]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key= os.environ["OPENAI_API_KEY"]  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

In [24]:
# Cypher generation prompt
cypher_generation_template = """
You are an expert Neo4j Cypher translator who converts English to Cypher based on the Neo4j Schema provided, following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE, HAVING keywords in the cypher. Use alias when using the WITH keyword
3. Use only Nodes and relationships mentioned in the schema
4. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Client, use `toLower(client.id) contains 'neo4j'`. To search for Slack Messages, use 'toLower(SlackMessage.text) contains 'neo4j'`. To search for a project, use `toLower(project.summary) contains 'logistics platform' OR toLower(project.name) contains 'logistics platform'`.)
5. Never use relationships that are not mentioned in the given schema
6. When asked about projects, Match the properties using case-insensitive matching and the OR-operator, E.g, to find a logistics platform -project, use `toLower(project.summary) contains 'logistics platform' OR toLower(project.name) contains 'logistics platform'`.

schema: {schema}

Examples:
Question: Which client's projects use most of our people?
Answer: ```MATCH (c:CLIENT)<-[:HAS_CLIENT]-(p:Project)-[:HAS_PEOPLE]->(person:Person)
RETURN c.name AS Client, COUNT(DISTINCT person) AS NumberOfPeople
ORDER BY NumberOfPeople DESC```
Question: Which person uses the largest number of different technologies?
Answer: ```MATCH (person:Person)-[:USES_TECH]->(tech:Technology)
RETURN person.name AS PersonName, COUNT(DISTINCT tech) AS NumberOfTechnologies
ORDER BY NumberOfTechnologies DESC```

Question: {question}
"""

cypher_prompt = PromptTemplate(
    template = cypher_generation_template,
    input_variables = ["schema", "question"]
)

In [26]:
CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
The information part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
If the provided information is empty, say that you don't know the answer.
Final answer should be easily readable and structured.
Information:
{context}

Question: {question}
Helpful Answer:"""

qa_prompt = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

In [27]:
def query_graph(user_input):
    graph = Neo4jGraph(url=neo4j_url, username=neo4j_user, password=neo4j_password)
    chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        verbose=True,
        return_intermediate_steps=True,
        cypher_prompt=cypher_prompt,
        qa_prompt=qa_prompt
        )
    result = chain(user_input)
    return result

In [29]:
user_input = "What is the relationship between Tanith and Lord Rykard?"

result = query_graph(user_input)
intermediate_steps = result["intermediate_steps"]
cypher_query = intermediate_steps[0]["query"]
database_results = intermediate_steps[1]["context"]
answer = result["result"]

print(answer)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (e1:Entity {name: 'Tanith'})-[r:RELATED]-(e2:Entity {name: 'Lord Rykard'})
RETURN e1.name AS Entity1, type(r) AS RelationshipType, r.description AS Description, r.metadata AS Metadata, r.order AS Order, e2.name AS Entity2

Full Context:
[{'Entity1': 'Tanith', 'RelationshipType': 'RELATED', 'Description': "Tanith continued to stay by Lord Rykard's side.", 'Metadata': '{"summary": "Tanith chose to remain with Lord Rykard despite receiving the Tonic of Forgetfulness.", "generated_at": "2024-08-24 20:26:22.238546"}', 'Order': 329, 'Entity2': 'Lord Rykard'}, {'Entity1': 'Tanith', 'RelationshipType': 'RELATED', 'Description': "Tanith stayed by Lord Rykard's side even after his men turned against him.", 'Metadata': '{"summary": "Tanith remained loyal to Lord Rykard despite his men betraying him.", "generated_at": "2024-08-24 20:26:22.238546"}', 'Order': 328, 'Entity2': 'Lord Rykard'}, {'Entity1': 'Tanith', 'Relationsh

In [12]:
print(type(result))
print(result)
print(answer)

<class 'dict'>
{'query': 'What is the relationship between Bilbo ang Gollum?', 'result': 'The relationship between Bilbo and Gollum is that the Ring passed from Gollum to Bilbo.', 'intermediate_steps': [{'query': "cypher\nMATCH (e1:Entity {name: 'Bilbo'})-[r:RELATED]-(e2:Entity {name: 'Gollum'})\nRETURN e1.name AS Entity1, e2.name AS Entity2, r.description AS RelationshipDescription, r.metadata AS RelationshipMetadata, r.order AS RelationshipOrder\n"}, {'context': [{'Entity1': 'Bilbo', 'Entity2': 'Gollum', 'RelationshipDescription': 'The Ring passed from Gollum to Bilbo', 'RelationshipMetadata': '{"summary": "Bilbo Baggins celebrates his birthday and leaves the Ring to Frodo, who is advised by Gandalf to take it away from the Shire. Frodo, along with companions Merry, Pippin, and Sam, embark on a journey to Rivendell while being pursued by Black Riders. They are aided by various characters including Tom Bombadil and Strider (Aragorn), and eventually reach Rivendell where they learn mor